# Prova de Python para Análise de Dados - EPGE - 2022
---

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Questão 1

Crie uma função $f$ que receba um número $n$ e retorne

$$
f(n) = \sqrt{6\cdot\sum_{i=1}^{n} \frac{1}{i^2}}
$$

Obs.:
1. Assuma que $n$ pode ser tanto um _int_ quanto um _float_ mas nada além disso.
2. Sua função deve ser capaz de calcular $f(10^8)$ em menos de 1,5s.

In [ ]:
import numpy as np

def f(n):
    x = np.arange(1,n+1)
    x = 1/x**2
    return(np.sqrt(6 * x.sum()))

In [ ]:
#Esta célula calcula quanto tempo sua função demora para rodar f(10^8).
#Deixe esta célula como a última célula da sua resposta.
%timeit f(1e8)

# Questão 2

Importe os dados sobre munocípios brasileiros que você encontra na url abaixo:

In [ ]:
url = "https://raw.githubusercontent.com/mapaslivres/municipios-br/main/tabelas/municipios.csv"

Você deve importar _apenas_ as colunas `municipio`, e `pop_21`, que são as colunas que iremos usar.

A variável `município` é o código IBGE do município. O primeiro dígito dessa variável representa a região do Brasil em que o município se encontra:

* 1 é região Norte
* 2 é região Nordeste
* 3 é região Sudeste
* 4 é região Sul
* 5 é região Centro-Oeste

Com esses dados, construa um dataframe que contenha a população total de cada região do Brasil, em ordem decrescente de população.

Esse dataframe deve ser indexado pelas regiões do Brasil, denotadas N, NE, SE, S e CO.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(url, usecols=['municipio','pop_21'])

In [ ]:
df['regiao'] = (df.
                municipio.
                apply(lambda x: str(x)[0]).
                values
               )

df['regiao'] = df.regiao.replace({'1': 'N', '2':'NE','3':'SE','4':'S','5':'CO'})

In [ ]:
df[['regiao','pop_21']].groupby('regiao').sum().sort_values('pop_21', ascending=False)

# Questão 3

Instale o pacote `investpy` e utilize a função `get_stock_historical_data` para buscar dados da ação `petr4` desde 01/01/2020 até ontem.

Nesta questão, sempre que falarmos no "preço" da ação, estamos nos referindo ao preço médio da ação no dia, definido por

$$
\frac{Open + High + Low + Close}{4}
$$

(existem outras definições, mas vamos usar esta, por simplicidade).

Além disso, vamos usar o retorno diário, definido como a variaçao percentual do preço médio de um dia para o outro.

## (a)

Faça um gráfico que mostre a evolução do preço da ação ao longo do tempo.  

Esse gráfico deve ser interativo, de modo que seja possível colocar o cursor sobre o gráfico e ver a data e o valor da ação naquela data.

In [ ]:
from investpy import get_stock_historical_data 
df = get_stock_historical_data('petr4', 'brazil', from_date='01/01/2020', to_date='28/7/2022')
df['preco_medio'] = (df.Open + df.High + df.Low + df.Close)/4

In [ ]:
df.preco_medio.plot(backend='plotly')

## (b)

Faça uma figura contendo 4 gráficos, distribuídos em duas linhas e duas colunas.  

Os gráficos devem ser os seguintes:

* No canto superior esquerdo, a série de preços. Junto com a série de preços, devemos também ver a média móvel de 50 dias em vermelho.

* No canto superior direito, a série de retornos diários da ação. Conforme antecipado anteriormente, o retorno diário consiste na variação percentual do preço da ação de um dia para o outro.

* No canto inferior esquerdo, a função de autocorrelação do preço da ação.

* No canto inferior direito, a função de autocorrelação parcial do preço da ação.

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
df.preco_medio.plot()
df.preco_medio.rolling(50).mean().plot(c='red', ls='dashed')

plt.subplot(2,2,2)
df.preco_medio.pct_change().plot()


plt.subplot(2,2,3)
pd.Series(sm.tsa.acf(df.preco_medio)).plot(kind='bar')

plt.subplot(2,2,3)
pd.Series(sm.tsa.acf(df.preco_medio)).plot(kind='bar')

plt.subplot(2,2,4)
pd.Series(sm.tsa.pacf(df.preco_medio)).plot(kind='bar')

plt.show()

## (c)

Faça um reste de Shapiro-Wilk para testar a hipótese de que os retornos diários seguem uma distribuição normal. Salve o valor-p desse teste como uma variável chamada `p` e dè um `print` de seu valor.

Para fins desse exercício, em caso de dados faltantes, você pode simplesmente retirá-los da série.

In [ ]:
from scipy.stats import shapiro
_, p = shapiro(df.preco_medio.pct_change().dropna())
print(p)

# Questão 4

O link a seguir contém dados a respeito do processo seletivo de uma empresa. As variáveis são as seguintes:

* `gender`: O gênero do candidato
* `sat`: A nota tirada pelo candidato no exame de SAT.
* `gpa`: O C.R. do candidato
* `apttest`: A nota que o candidato obteve em um teste de aptidão à vaga
* `int1` : A nota dada ao candidato pelo primeiro entrevistador
* `int2` : A nota dada ao candidato pelo segundo entrevistador
* `int3` : A nota dada ao candidato pelo terceiro entrevistador
* `hired`: 1, caso o candidato tenha sido contratado, e 0 caso contrário.



In [ ]:
url = "https://peopleanalytics-regression-book.org/data/recruiting.csv"

## (a) 
Qual fração dos candidatos são mulheres?  
Qual fração dos aprovados são mulheres?

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.gender.value_counts(normalize=True)

In [ ]:
df[['gender','hired']].replace({'F': 1, 'M': 0}).groupby('hired').mean()

* 51% dos candidatos são mulheres.  
* 42% dos aprovados são mulheres.

## (b)
Faça uma regressão logística para explicar a contratação (`hired`) em termos das demais variáveis da base. Use uma abordagem de Estatística clássica, produzindo estimativas para os coeficientes, erros-padrão e p-valores.  

Exiba a tabela com os resultados da regressão.

Obs.: A regressão deve ter uma única dummy designando gênero e esta deve ser 1 quando o gênero é masculino e 0 caso contrário.

In [ ]:
dfreg = pd.get_dummies(df).drop(columns='gender_F')

X = dfreg.drop(columns='hired')
y = dfreg['hired']
X = sm.add_constant(X)

lg = sm.Logit(y,X).fit()
print(lg.summary())

# Questão 5

Utilizando os mesmos dados da questão anterior, construa um modelo de Data Science para prever a `hired` a partir das outras variáveis (_features_). 

Aqui está novamente o link, para facilitar:

In [ ]:
url = "https://peopleanalytics-regression-book.org/data/recruiting.csv"

Antes de mais nada, normalize as variáveis dividindo-as todas pelos seus valores máximos:
* o valor máximo do `sat` é 1600
* o valor máximo do `gpa` é 4
* o valor máximo do `apttest` é 100
* os valores máximos de `int1`, `int2` e `int3` são todos iguais a 5
Após a normalização, todas essas variáveis deverão estar contidas no intervalo $[0;1]$.

A seguir, cria um dataframe contendo _precision_, _recall_, _f1_, _accuracy_ e _AUC_ para os modelos a seguir. Para facilitar, eu já disponibilizo o código para importá-los:

In [ ]:
# Modelos
from sklearn.dummy import DummyClassifier #dummy
from sklearn.linear_model import LogisticRegression #reg_log
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.tree import DecisionTreeClassifier #arvore_decisao
from sklearn.ensemble import RandomForestClassifier #random_forest

#Métricas
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score

#Utils
from sklearn.model_selection import train_test_split

## (a)
Apresente esse dataframe ordenado em ordem decrescente de AUC.

In [ ]:
df = pd.read_csv(url)

df['gender'] = df.gender.replace({'F': 0, 'M': 1})
df['sat'] = df.sat/1600
df['gpa'] = df.gpa/4
df['apttest'] = df.apttest/100
df['int1'] = df.int1/5
df['int2'] = df.int2/5
df['int3'] = df.int3/5

df = df.astype('float64')

In [ ]:
modelos = {'dummy': DummyClassifier(),
           'logistic': LogisticRegression(),
           'KNN': KNeighborsClassifier(),
           'tree': DecisionTreeClassifier(),
           'rf': RandomForestClassifier()
          }


resultados = pd.DataFrame([], index = modelos.keys(), columns=['precision','recall','f1','accuracy','auc'])

In [ ]:
X = df.drop(columns='hired')
y = df.hired

In [ ]:
Xtrn, Xtst ,ytrn,  ytst = train_test_split(X,y)

In [ ]:
plt.figure(figsize=(10,10))

for nome, modelo in modelos.items():
    modelo.fit(Xtrn,ytrn)
    ypred = modelo.predict(Xtst)
    ppred = modelo.predict_proba(Xtst)[:,1]
    

    
    precision = precision_score(ytst, ypred)
    recall = recall_score(ytst, ypred)
    f1 = f1_score(ytst, ypred)
    accuracy = accuracy_score(ytst, ypred)
    auc = roc_auc_score(ytst, ppred)
    
    resultados.loc[nome,:] = precision, recall, f1, accuracy, auc
    
    fpr, tpr, _ = roc_curve(ytst, ppred)
    plt.plot(fpr, tpr, label=f'{nome} ({auc:.2f})')
    
    modelos[nome] = modelo

plt.title('Curvas ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0,1],[0,1], c='gray', linestyle='dashed', label='aleatório')
plt.legend()
plt.axis('square')
plt.show()

In [ ]:
resultados.sort_values('auc', ascending=False)

## (b)
Apresente as curvas ROC de cada modelo em um único gráfico. O gráfico deve ter:

* Título: "Curvas ROC"
* Nome do eixo x: "Taxa de falsos positivos"
* Nome do eixo y: "Taxa de verdadeiros positivos"
* Tamanho 10 x 10
* Legenda, indicando qual curva corresponde a qual modelo, com os valores dos respectivos AUC entre parêntesis. (Por exemplo, "dummy (0.54)")
* Uma reta cinza tracejada entre os pontos (0,0) e (1,1) e legenda "aleatório"

Você pode responder essa pergunta juntamente com a anterior, se preferir.

## (c)

Salve o modelo com o melhor AUC em um arquivo. Em seguida, importe o modelo deste arquivo com um nome diferente do nome do seu modelo inicial. 

In [ ]:
import pickle

In [ ]:
pickle.dump(modelos['rf'], open('meu_modelo.pkl','wb'))

In [ ]:
minha_rf = pickle.load(open('meu_modelo.pkl','rb'))

In [ ]:
minha_rf